## Obtaining data from Soccerbase 

In [2]:
import pandas as pd
import requests
import numpy as np
import re
from bs4 import BeautifulSoup as bs
from IPython.core.display import display,HTML

In [69]:
url = 'https://www.soccerbase.com/teams/home.sd'
r = requests.get(url)
soup = bs(r.content, 'html.parser')
teams = soup.find('div', {'class': 'headlineBlock'}, text='Team').next_sibling.find_all('li') #PL APENAS!

- The cell above this one just find the Premier League teams
- So, we just search on html code the name of top five european leagues
- After that, we put them into a dictionary and associate the competition id to clubs path at soccerbase

In [62]:
ligas = soup.find_all(href=re.compile('tournaments/tournament')) #NOME DE TODAS AS LIGAS NO DATASET
ligas_dict = {}
idleague = {}
for liga in ligas:
    linky = 'https://www.soccerbase.com' + liga['href']
    idz = liga['href'][27:] #[35:] para só numero
    liga = liga.text
    ligas_dict[liga] = linky
    idleague[liga] =  idz
    
#ligas_dict
#idleague

In [73]:
def extract_data(teams_dict):
    team = []
    comps = []
    dates = []
    h_teams = []
    a_teams = []
    h_scores = []
    a_scores = []

    consolidated = []
    for k, v in teams_dict.items():
        print('Acquiring %s data...' % k)

        headers = ['Team', 'Competition', 'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Date Keep']
        r = requests.get('%s&teamTabs=results' % v)
        soup = bs(r.content, 'html.parser')

        h_scores.extend([int(i.text) for i in soup.select('.score a em:first-child')])
        limit_scores = [int(i.text) for i in soup.select('.score a em + em')]
        a_scores.extend([int(i.text) for i in soup.select('.score a em + em')])

        limit = len(limit_scores)
        team.extend([k for i in soup.select('.tournament', limit=limit)])
        comps.extend([i.text for i in soup.select('.tournament a', limit=limit)])
        dates.extend([i.text for i in soup.select('.dateTime .hide', limit=limit)])
        h_teams.extend([i.text for i in soup.select('.homeTeam a', limit=limit)])
        a_teams.extend([i.text for i in soup.select('.awayTeam a', limit=limit)])



    df = pd.DataFrame(list(zip(team, comps, h_teams, h_scores, a_teams, a_scores, dates)),
                          columns=headers)
    return df

- We pass a list with the leagues of interest
- So, we must to have a dictionary of dictionaries, where the leagues points to a dataframe with clubs matches at this point of the season  

In [84]:
Top_Ligas = ['Premier League','Italian Serie A','German Bundesliga','Spanish La Liga','French Ligue 1']
top_dict = {}
df_dict = {}
for lg in Top_Ligas:
    indicador = idleague[lg]
    srateams = soup.find_all(href=re.compile(indicador))
    dictimes = {}
    for i in range(len(srateams)):
        if indicador not in srateams[i].attrs['href'][:-1] and 'teams' in srateams[i].attrs['href'][:-1]:
            tm = srateams[i].text
            lk = 'https://www.soccerbase.com' + srateams[i]['href']
            dictimes[tm] = lk
            
    top_dict[lg] = dictimes
    data = extract_data(top_dict[lg])
    df_dict[lg] = data

#top_dict - DICIONARIO DE TIMES
#df_dict - DICIONARIO DE DATAFRAMES

Acquiring Arsenal data...
Acquiring Aston Villa data...
Acquiring Brighton data...
Acquiring Burnley data...
Acquiring Chelsea data...
Acquiring Crystal Palace data...
Acquiring Everton data...
Acquiring Fulham data...
Acquiring Leeds data...
Acquiring Leicester data...
Acquiring Liverpool data...
Acquiring Man City data...
Acquiring Man Utd data...
Acquiring Newcastle data...
Acquiring Sheff Utd data...
Acquiring Southampton data...
Acquiring Tottenham data...
Acquiring West Brom data...
Acquiring West Ham data...
Acquiring Wolves data...
Acquiring Atalanta data...
Acquiring Benevento data...
Acquiring Bologna data...
Acquiring Cagliari data...
Acquiring Crotone data...
Acquiring Fiorentina data...
Acquiring Genoa data...
Acquiring Inter data...
Acquiring Juventus data...
Acquiring Lazio data...
Acquiring Milan data...
Acquiring Napoli data...
Acquiring Parma data...
Acquiring Roma data...
Acquiring Sampdoria data...
Acquiring Sassuolo data...
Acquiring Spezia data...
Acquiring Torino

{'Premier League':         Team       Competition    Home Team  Home Score  Away Team  \
 0    Arsenal  Community Shield      Arsenal           1  Liverpool   
 1    Arsenal    Premier League       Fulham           0    Arsenal   
 2    Arsenal    Premier League      Arsenal           2   West Ham   
 3    Arsenal           EFL Cup    Leicester           0    Arsenal   
 4    Arsenal    Premier League    Liverpool           3    Arsenal   
 ..       ...               ...          ...         ...        ...   
 761   Wolves    Premier League       Wolves           1      Leeds   
 762   Wolves    Premier League    Newcastle           1     Wolves   
 763   Wolves    Premier League     Man City           4     Wolves   
 764   Wolves    Premier League  Aston Villa           0     Wolves   
 765   Wolves    Premier League       Wolves           0  Liverpool   
 
      Away Score         Date Keep  
 0             1  2020-08-29 16:30  
 1             3  2020-09-12 12:30  
 2             1 

In [86]:
def classifica(df,nome_liga): #dicionario de dataframes
    vec = np.unique(df['Team'])
    gplist = []; gclist = []; vlist = []; dlist = []; elist=[]; plist = []; slist=[]; jlist=[];
    dicio = {}
    ntimes = []
    for j in range(len(vec)):
        vit = 0; der = 0; emp = 0; gols_p = 0; gols_c = 0; pts = 0; saldo = 0; jogos = 0;
        data = df[df.loc[:,'Team'] == vec[j]].reset_index()
        datac = data[data.loc[:,'Competition'] == nome_liga].reset_index()
        jogos = len(datac)
        ntimes.append(j+1)
        for i in range(len(datac)):
            if datac.loc[i,'Home Team'] == vec[j]:
                gols_p += int(datac.loc[i,'Home Score'])
                gols_c += int(datac.loc[i,'Away Score'])
                if datac.loc[i,'Home Score'] > datac.loc[i,'Away Score']:
                    vit += 1
                elif datac.loc[i,'Home Score'] < datac.loc[i,'Away Score']:
                    der += 1
                else:
                    emp += 1
            elif datac.loc[i,'Away Team'] == vec[j]:
                gols_p += int(datac.loc[i,'Away Score'])
                gols_c += int(datac.loc[i,'Home Score'])
                if datac.loc[i,'Home Score'] < datac.loc[i,'Away Score']:
                    vit += 1
                elif datac.loc[i,'Home Score'] > datac.loc[i,'Away Score']:
                    der += 1
                else:
                    emp += 1
            else:
                gols_p +=0
                gols_c +=0
                
        pts = 3*vit + emp
        saldo = gols_p - gols_c
        gplist.append(gols_p); gclist.append(gols_c); vlist.append(vit); dlist.append(der); elist.append(emp); plist.append(pts)
        slist.append(saldo); jlist.append(jogos)
    
    dicio = {'Times': vec, 'Pontos': plist, 'Jogos': jlist, 'Vitórias': vlist, 'Empates': elist, 'Derrotas': dlist,
            'Gols Pró': gplist, 'Gols Contra': gclist, 'SG': slist, 'Posição': ntimes}
    classificacao = pd.DataFrame(dicio)
    return classificacao.sort_values(by=['Pontos'], ascending=False).set_index(classificacao['Posição']).drop(columns='Posição')

- classifica function returns a dataframe with the position of teams, sorted by the points at this time of the season
- Next, we pass the dictionary of dataframes and league names to obtain one dictionary with league tables

In [91]:
dict_tabs = {}
for lig in Top_Ligas:
    dadinho = classifica(df_dict[lig],lig)
    dict_tabs[lig] = dadinho

In [155]:
dict_tabs['Spanish La Liga']

,Times,Pontos,Jogos,Vitórias,Empates,Derrotas,Gols Pró,Gols Contra,SG
Posição,,,,,,,,,
1,Atl Madrid,66,29,20,6,3,51,19,32
2,Real Madrid,63,29,19,6,4,51,23,28
3,Barcelona,62,28,19,5,4,67,24,43
4,Sevilla,58,29,18,4,7,40,21,19
5,Villarreal,46,29,11,13,5,42,31,11
6,Real Betis,46,29,14,4,11,39,42,-3
7,Sociedad,45,28,12,9,7,44,28,16
8,Celta Vigo,37,29,9,10,10,38,44,-6
9,Granada,36,29,10,6,13,34,49,-15


In [126]:
dict_ligpos = {'Premier League': {'UCL':[1,2,3,4],'Pré-UCL':[None],'UEL':[5],'Pré-Reb':[None],'Reb':[18,19,20]},
               'Italian Serie A': {'UCL':[1,2,3,4],'Pré-UCL':[None],'UEL':[5],'Pré-Reb':[None],'Reb':[18,19,20]},
               'German Bundesliga': {'UCL':[1,2,3,4],'Pré-UCL':[None],'UEL':[5],'Pré-Reb':[18],'Reb':[19,20]},
               'Spanish La Liga': {'UCL':[1,2,3,4],'Pré-UCL':[None],'UEL':[5],'Pré-Reb':[None],'Reb':[18,19,20]},
               'French Ligue 1': {'UCL':[1,2],'Pré-UCL':[3],'UEL':[4],'Pré-Reb':[18],'Reb':[19,20]}}

- Lastly, we could to put the teams logos at dataframes, making them one html object

In [285]:
escudos = {'Man City': 'https://logodownload.org/wp-content/uploads/2017/02/manchester-city-fc-logo-escudo-badge-2.png',
           'Man Utd': 'https://logodetimes.com/times/manchester-united/logo-manchester-united-256.png',
           'Leicester': 'https://logodownload.org/wp-content/uploads/2019/05/leicester-city-logo-1.png',
           'Chelsea': 'https://logodownload.org/wp-content/uploads/2017/02/chelsea-fc-logo-escudo.png',
           'West Ham': 'https://logodownload.org/wp-content/uploads/2019/05/west-ham-united-logo.png',
           'Tottenham': 'https://logodetimes.com/times/tottenham/logo-tottenham-256.png',
           'Liverpool': 'https://logodownload.org/wp-content/uploads/2017/02/liverpool-fc-logo-escudo-8.png',
           'Everton': 'https://logodownload.org/wp-content/uploads/2019/04/everton-logo-escudo.png',
           'Arsenal': 'https://logodownload.org/wp-content/uploads/2017/02/Arsenal-logo-escudo-shield-1.png',
           'Aston Villa': 'https://logodownload.org/wp-content/uploads/2019/10/aston-villa-logo.png',
           'Leeds': 'https://s.yimg.com/cv/apiv2/default/soccer/20181205/500x500/LeedsUnited_wbg.png',
           'Crystal Palace': 'https://logodownload.org/wp-content/uploads/2019/05/crystal-palace-logo.png',
           'Wolves': 'https://logodownload.org/wp-content/uploads/2019/04/wolverhampton-logo-escudo.png',
           'Southampton': 'https://logodownload.org/wp-content/uploads/2019/10/southampton-fc-logo.png',
           'Burnley': 'https://logodownload.org/wp-content/uploads/2019/10/burnley-logo.png',
           'Brighton': 'https://logodownload.org/wp-content/uploads/2019/10/brighton-hove-albion-logo.png',
           'Newcastle': 'https://logodownload.org/wp-content/uploads/2019/10/newcastle-united-logo.png',
           'Fulham': 'https://upload.wikimedia.org/wikipedia/pt/thumb/1/13/Fulham_FC.svg/340px-Fulham_FC.svg.png',
           'West Brom': 'https://marcas-logos.net/wp-content/uploads/2020/03/WEST-BROMWICH-ALBION-LOGO.png',
           'Sheff Utd': 'https://logodownload.org/wp-content/uploads/2019/10/sheffield-united-logo.png'}

tabcop['Escudos'] = tabcop.Times.map(escudos)
cols = tabcop.columns.tolist()
cols = cols[-1:] + cols[:-1]
tabcop = tabcop[cols]

def path_to_image_html(path):
    return '<img src="'+ path + '" width="20" >'

pd.set_option('display.max_colwidth', None)

display(HTML(tabcop.to_html(escape=False ,formatters=dict(Escudos=path_to_image_html))))

,Escudos,Times,Pontos,Jogos,Vitórias,Empates,Derrotas,Gols Pró,Gols Contra,SG
Posição,,,,,,,,,,
1,,Man City,71,30,22,5,3,64,21,43
2,,Man Utd,57,29,16,9,4,56,32,24
3,,Leicester,56,29,17,5,7,53,32,21
4,,Chelsea,51,29,14,9,6,44,25,19
5,,West Ham,49,29,14,7,8,45,35,10
6,,Tottenham,48,29,14,6,9,49,30,19
7,,Liverpool,46,29,13,7,9,48,36,12
8,,Everton,46,28,14,4,10,40,37,3
9,,Arsenal,42,29,12,6,11,40,32,8
